<a href="https://colab.research.google.com/github/1Englert1/cerf_classification/blob/main/dev/Experimente.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Doku für CoreNLP unter Stanza**
https://stanfordnlp.github.io/stanza/client_properties.html#switching-language

#CoreNLP

##Set up CoreNLP with Stanza

In [ ]:
!pip install Stanza

     |████████████████████████████████| 235kB 4.4MB/s 


In [ ]:
import stanza
stanza.install_corenlp(dir="/content/corenlp")

2020-10-19 12:26:21 INFO: Installing CoreNLP package into /content/corenlp...
2020-10-19 12:26:44 WARNING: For customized installation location, please set the `CORENLP_HOME` environment variable to the location of the installation. In Unix, this is done with `export CORENLP_HOME=/content/corenlp`.


In [ ]:
import os
os.environ["CORENLP_HOME"] = "./corenlp"

In [ ]:
stanza.download_corenlp_models(model='german', version='4.1.0', dir="/content/corenlp")

2020-10-19 12:26:44 INFO: Downloading german models (version 4.1.0) into directory /content/corenlp...


##Pattern matching

In [ ]:
from stanza.server import CoreNLPClient

In [ ]:

client = CoreNLPClient(properties='german',
                   annotators='tokenize,ssplit,mwt,pos,parse',
                   timeout=300000,
                   output_format='text', 
                   be_quiet=True, 
                   endpoint='http://localhost:9001',
                   memory='8G')


2020-10-19 12:27:07 INFO: Using Stanford CoreNLP default properties for: german.  Make sure to have german models jar (available for download here: https://stanfordnlp.github.io/CoreNLP/) in CLASSPATH


In [ ]:
client.start()
import time; time.sleep(10)

2020-10-19 12:32:46 INFO: Starting server with command: java -Xmx8G -cp ./corenlp/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9001 -timeout 300000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties german -annotators tokenize,ssplit,mwt,pos,parse -preload -outputFormat text


In [ ]:
text1 = "Hast du dir schon überlegt, ob du morgen zur Deutschstunde gehst?"
text2 = "Was machst du stattdessen, wenn du nicht zur Deutschstunde gehst?"
text3 = "Wenn ich nicht zur Deutschstunde gehe, bleibe ich im Bett liegen."
text4 = "Peter kommt und Paul geht."
text5 = "Wir gehen morgen, falls es nicht regnet, in den Zoo."
text6 = "Wird das Wetter morgen besser, gehen wir ins Freibad."


In [ ]:
def match_pattern(_pattern, _text):
  matches = client.tregex(_text,_pattern)
  list = matches['sentences']
  for sentence in matches['sentences']:
    for match_id in sentence:
      print(sentence[match_id]['spanString'])

def len_sum_matches(_pattern, _text):
  matches = client.tregex(_text,_pattern)
  list = matches['sentences']
  match_list = []
  for sentence in matches['sentences']:
    for match_id in sentence:
      match_list.append(sentence[match_id]['spanString'])
  len_sum = 0
  for item in match_list:
    len_sum = len_sum + len(item)
  return(len_sum)

def count_matches(_pattern, _text):
  matches = client.tregex(_text,_pattern)
  list = matches['sentences']
  count_units = 0
  for sentence in matches['sentences']:
    for match_id in sentence:
      count_units = count_units + 1
  return(count_units)

In [ ]:
texts = [text1, text2, text3, text4, text5, text6]
for text in texts:
  print("################Neuer Text#####################")
  print("################Text#####################")
  print(text)
  print("################Tree#####################")
  ann = client.annotate(text)
  print(ann.split("parse:",1)[1])

  ################test#####################
  print("\n###############Test###################")
  pattern = "CAC|CAVP|CNP|CVP"
  match_pattern(pattern, text)
  print(count_matches(pattern, text))


  ################T-Units#####################
  print("\n###############T-Units###################")
  pattern = "(S > NUR) | (S >> CS >> NUR)"
  match_pattern(pattern, text)
  print(count_matches(pattern, text))

  ################Complexe T-Units#####################
  print("\n###############Complex T-Units###################")
  pattern = "S >> (S > (CS > NUR > ROOT) | > (NUR > ROOT)) "
  match_pattern(pattern, text)
  print(count_matches(pattern, text))

################Neuer Text#####################
################Text#####################
Hast du dir schon überlegt, ob du morgen zur Deutschstunde gehst?
################Tree#####################
 
(ROOT
  (NUR
    (S (VERB Hast) (PROPN du) (PRON dir) (ADV schon)
      (VP (VERB überlegt) (PUNCT ,)
        (S (SCONJ ob) (PROPN du) (ADV morgen)
          (VP
            (PP (ADP zu) (DET der) (NOUN Deutschstunde))
            (VERB gehst)))))
    (PUNCT ?)))



###############Test###################
0

###############T-Units###################
Hast du dir schon überlegt, ob du morgen zu der Deutschstunde gehst
1

###############Complex T-Units###################
ob du morgen zu der Deutschstunde gehst
1
################Neuer Text#####################
################Text#####################
Was machst du stattdessen, wenn du nicht zur Deutschstunde gehst?
################Tree#####################
 
(ROOT
  (NUR
    (S (PRON Was) (VERB machst) (PROPN du) (ADV stattdessen) (PUNCT ,)
  

In [ ]:
client.stop()

time.sleep(10)
!ps -o pid,cmd | grep java

    180 /bin/bash -c ps -o pid,cmd | grep java
    182 grep java


#Berkeley Neural Parser

##Installation

In [ ]:
!pip install spacy

In [ ]:
!pip install benepar

     |████████████████████████████████| 81kB 3.9MB/s 
  Created wheel for benepar: filename=benepar-0.1.2-cp36-cp36m-linux_x86_64.whl size=104471 sha256=3c560ccd9acfc97b34e20894eaaaa923980d4548d98f258dd73c9ca0cb5efd50
  Stored in directory: /root/.cache/pip/wheels/c6/f5/06/d88543b19a9b326007d7538298a139e994b1d2eecb003bf5af
Successfully built benepar


In [ ]:
!pip install tensorflow==1.14.0

     |████████████████████████████████| 109.2MB 98kB/s 
     |████████████████████████████████| 51kB 7.3MB/s 
     |████████████████████████████████| 3.2MB 44.4MB/s 
     |████████████████████████████████| 491kB 42.7MB/s 
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninstalled tensorboard-2.3.0
  Found existing installation: tensorflow-estimator 2.3.0
    Uninstalling tensorflow-estimator-2.3.0:
      Successfully uninstalled tensorflow-estimator-2.3.0
  Found existing installation: tensorflow 2.3.0
    Uninstalling tensorflow-2.3.0:
      Successfully uninstalled tensorflow-2.3.0


In [ ]:
!pip install textacy

     |████████████████████████████████| 184kB 4.5MB/s 
     |████████████████████████████████| 481kB 9.6MB/s 
     |████████████████████████████████| 3.0MB 12.5MB/s 
     |████████████████████████████████| 102kB 12.6MB/s 
  Created wheel for cytoolz: filename=cytoolz-0.11.0-cp36-cp36m-linux_x86_64.whl size=1225604 sha256=b6974c65d0f1745eb9807bb01d2f229635be7071fe9863e0cfa451e9e0b8cad7
  Stored in directory: /root/.cache/pip/wheels/a1/32/3c/9c9926b510647cacdde744b2c7acdf1ccd5896fbb7f8d5df0c
Successfully built cytoolz


In [ ]:
!pip install nltk_tgrep

In [ ]:
import spacy
from benepar.spacy_plugin import BeneparComponent
import nltk
import benepar
import textacy
import nltk_tgrep
from nltk.tree import ParentedTree

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
!python -m spacy download de
benepar.download('benepar_de')

     |████████████████████████████████| 14.9MB 774kB/s 
  Created wheel for de-core-news-sm: filename=de_core_news_sm-2.2.5-cp36-none-any.whl size=14907056 sha256=a4dc655f0ecff18c8677eaaf6f5db8d2e8b5eae290012f1bd569b085f841fa46
  Stored in directory: /tmp/pip-ephem-wheel-cache-y14r70gz/wheels/ba/3f/ed/d4aa8e45e7191b7f32db4bfad565e7da1edbf05c916ca7a1ca
Successfully built de-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/de_core_news_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/de
You can now load the model via spacy.load('de')
[nltk_data] Downloading package benepar_de to /root/nltk_data...


True

##Pattern matching

In [ ]:
text = "Hast du dir schon überlegt, ob du morgen zur Deutschstunde gehst?"
text2 = "Was machst du stattdessen, wenn du nicht zur Deutschstunde gehst?"
text3 = "Wenn ich nicht zur Deutschstunde gehe, bleibe ich im Bett liegen."
text4 = "Peter kommt und Paul geht."

In [ ]:
parser = benepar.Parser("benepar_de")
tree = parser.parse(text)
print(tree)



(VROOT
  (S
    (VAFIN-HD Hast)
    (PPER-SB du)
    (VP-OC (PPER-DA dir) (ADV-MO schon) (VVPP-HD überlegt)))
  ($, ,)
  (S-OC
    (KOUS-CP ob)
    (PPER-SB du)
    (ADV-MO morgen)
    (PP-MO (APPRART-AC zur) (NN-NK Deutschstunde))
    (VVFIN-HD gehst))
  ($. ?))


In [ ]:
tree2 = parser.parse(text2)
print(tree2)

(VROOT
  (S
    (PWS-OA Was)
    (VVFIN-HD machst)
    (PPER-SB du)
    (PROAV-MO stattdessen))
  ($, ,)
  (S-MO
    (KOUS-CP wenn)
    (PPER-SB du)
    (PTKNEG-NG nicht)
    (PP-MO (APPRART-AC zur) (NN-NK Deutschstunde))
    (VVFIN-HD gehst))
  ($. ?))


In [ ]:
tree3 = parser.parse(text3)
print(tree3)

(VROOT
  (S-MO
    (KOUS-CP Wenn)
    (PPER-SB ich)
    (PTKNEG-NG nicht)
    (PP-MO (APPRART-AC zur) (NN-NK Deutschstunde))
    (VVFIN-HD gehe))
  ($, ,)
  (S
    (VVFIN-HD bleibe)
    (PPER-SB ich)
    (VP-OC (PP-MO (APPRART-AC im) (NN-NK Bett)) (VVINF-HD liegen)))
  ($. .))


In [ ]:
tree4 = parser.parse(text4)
print(tree4)

(VROOT
  (CS
    (S-CJ (NE-SB Peter) (VVFIN-HD kommt))
    (KON-CD und)
    (S-CJ (NE-SB Paul) (VVFIN-HD geht)))
  ($. .))


In [ ]:
pattern = 'VROOT < S'
match = nltk_tgrep.tgrep_nodes(tree, pattern)
print(match)
print(len(match))

[Tree('VROOT', [Tree('S', [Tree('VAFIN-HD', ['Hast']), Tree('PPER-SB', ['du']), Tree('VP-OC', [Tree('PPER-DA', ['dir']), Tree('ADV-MO', ['schon']), Tree('VVPP-HD', ['überlegt'])])]), Tree('$,', [',']), Tree('S-OC', [Tree('KOUS-CP', ['ob']), Tree('PPER-SB', ['du']), Tree('ADV-MO', ['morgen']), Tree('PP-MO', [Tree('APPRART-AC', ['zur']), Tree('NN-NK', ['Deutschstunde'])]), Tree('VVFIN-HD', ['gehst'])]), Tree('$.', ['?'])])]
1


In [ ]:
print(len(nltk_tgrep.tgrep_nodes(tree, pattern)))

1


In [ ]:
nlp = spacy.load('de')


In [ ]:
text = "Tom fragt Maria, ob sie ihm die Wahrheit sagt, aber sie antwortet nicht."
doc = textacy.make_spacy_doc(text)
doc._.preview

100%|██████████| 66.7M/66.7M [00:02<00:00, 29.2MB/s]
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator HashingVectorizer from version 0.22 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LabelBinarizer from version 0.22 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator MLPClassifier from version 0.22 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator Pipeline from version 0.22 when using version 0.22.2.post1. This migh

'Doc(16 tokens: "Tom fragt Maria, ob sie ihm die Wahrheit sagt, ...")'

In [ ]:
doc = nlp(text)
print(doc)

Tom fragt Maria, ob sie ihm die Wahrheit sagt, aber sie antwortet nicht.


In [ ]:
parser_de = benepar.Parser("benepar_de")
pattern = 'CS > /^S/'
for sent in sents:
  tree = parser_de.parse(str(sent))
  print(tree)
  print(len(nltk_tgrep.tgrep_nodes(tree, pattern)))

NameError: ignored

#Type/Token Ratio

In [1]:
!pip install spacy

In [18]:
!pip install textacy

     |████████████████████████████████| 184kB 2.9MB/s 
     |████████████████████████████████| 3.0MB 8.7MB/s 
     |████████████████████████████████| 481kB 30.2MB/s 
     |████████████████████████████████| 102kB 6.4MB/s 
  Created wheel for cytoolz: filename=cytoolz-0.11.0-cp36-cp36m-linux_x86_64.whl size=1225581 sha256=3839f3b6450949ac91a5ad137cf5bb5c6c94b35c977613661c7090d24a97f129
  Stored in directory: /root/.cache/pip/wheels/a1/32/3c/9c9926b510647cacdde744b2c7acdf1ccd5896fbb7f8d5df0c
Successfully built cytoolz


In [2]:
!python -m spacy download de

     |████████████████████████████████| 14.9MB 471kB/s 
  Created wheel for de-core-news-sm: filename=de_core_news_sm-2.2.5-cp36-none-any.whl size=14907056 sha256=db29db25c2ec655fe4e303e7d9f7be3b95835bffe1bcaff688e171acb8e4e573
  Stored in directory: /tmp/pip-ephem-wheel-cache-a39j7yc6/wheels/ba/3f/ed/d4aa8e45e7191b7f32db4bfad565e7da1edbf05c916ca7a1ca
Successfully built de-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/de_core_news_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/de
You can now load the model via spacy.load('de')


In [25]:
import spacy
from spacy.tokenizer import Tokenizer
from spacy.lang.de import German
nlp = German()

In [37]:
text = "sehr gefruet auf diser Platz. Deshalb möchte ich mich bewarben. Ich glaube es ist ein Richtung wo ich mich besser Bildet machen könnte. Ich habe auch Ihre Anzeige gelesen. Diese Anzeige gefällt mir so sehr Da gibt alles Information was brauche ich . Jetzt wollte ich meine ProsPeckte geben für Ihre Kenntnis. Ich habe Studium gemacht als Computer Science in Stadt Y in Indien. Denn habe ich zwei Jahre ein Job gehabt in Stadt Z und zuletzt arbeite ich von 01.04.2009 bis Jetzt ein Telekaffe. Ihre Anzeige ist für mich ein Gottest Geseck wenn Sie mich ein Platz riserviren könnte.Ich möchte gerne auch wissen ab wann Beginn dieser Ausbildung, wie viele Stunden Prowoche und Arbeitszeit. Ich möchte vorher Ihn Dankber wenn Sie mich züruck schreiben können. Viellen Dank und viellen Grüßen Frau MeierAnlage."
text = textacy.preprocessing.remove.remove_punctuation(text)
text = textacy.preprocessing.normalize.normalize_whitespace(text)
text = textacy.preprocessing.replace.replace_numbers(text)

print(text)

sehr gefruet auf diser Platz Deshalb möchte ich mich bewarben Ich glaube es ist ein Richtung wo ich mich besser Bildet machen könnte Ich habe auch Ihre Anzeige gelesen Diese Anzeige gefällt mir so sehr Da gibt alles Information was brauche ich Jetzt wollte ich meine ProsPeckte geben für Ihre Kenntnis Ich habe Studium gemacht als Computer Science in Stadt Y in Indien Denn habe ich zwei Jahre ein Job gehabt in Stadt Z und zuletzt arbeite ich von _NUMBER_ _NUMBER_ _NUMBER_ bis Jetzt ein Telekaffe Ihre Anzeige ist für mich ein Gottest Geseck wenn Sie mich ein Platz riserviren könnte Ich möchte gerne auch wissen ab wann Beginn dieser Ausbildung wie viele Stunden Prowoche und Arbeitszeit Ich möchte vorher Ihn Dankber wenn Sie mich züruck schreiben können Viellen Dank und viellen Grüßen Frau MeierAnlage


In [38]:
sentence = nlp(text)
for word in sentence:
  print(word.text)

sehr
gefruet
auf
diser
Platz
Deshalb
möchte
ich
mich
bewarben
Ich
glaube
es
ist
ein
Richtung
wo
ich
mich
besser
Bildet
machen
könnte
Ich
habe
auch
Ihre
Anzeige
gelesen
Diese
Anzeige
gefällt
mir
so
sehr
Da
gibt
alles
Information
was
brauche
ich
Jetzt
wollte
ich
meine
ProsPeckte
geben
für
Ihre
Kenntnis
Ich
habe
Studium
gemacht
als
Computer
Science
in
Stadt
Y
in
Indien
Denn
habe
ich
zwei
Jahre
ein
Job
gehabt
in
Stadt
Z
und
zuletzt
arbeite
ich
von
_
NUMBER
_
_
NUMBER
_
_
NUMBER
_
bis
Jetzt
ein
Telekaffe
Ihre
Anzeige
ist
für
mich
ein
Gottest
Geseck
wenn
Sie
mich
ein
Platz
riserviren
könnte
Ich
möchte
gerne
auch
wissen
ab
wann
Beginn
dieser
Ausbildung
wie
viele
Stunden
Prowoche
und
Arbeitszeit
Ich
möchte
vorher
Ihn
Dankber
wenn
Sie
mich
züruck
schreiben
können
Viellen
Dank
und
viellen
Grüßen
Frau
MeierAnlage


In [41]:
uniqueToken = []
for word in sentence:
  token = word.lemma_
  if token not in uniqueToken:
    uniqueToken.append(token)
print(uniqueToken)

['sehr', 'gefruet', 'auf', 'diser', 'Platz', 'Deshalb', 'möchte', 'ich', 'mich', 'bewarben', 'Ich', 'glaube', 'es', 'ist', 'ein', 'Richtung', 'wo', 'besser', 'Bildet', 'machen', 'könnte', 'habe', 'auch', 'Ihre', 'Anzeige', 'gelesen', 'Diese', 'gefällt', 'mir', 'so', 'Da', 'gibt', 'alles', 'Information', 'was', 'brauche', 'Jetzt', 'wollte', 'meine', 'ProsPeckte', 'geben', 'für', 'Kenntnis', 'Studium', 'gemacht', 'als', 'Computer', 'Science', 'in', 'Stadt', 'Y', 'Indien', 'Denn', 'zwei', 'Jahre', 'Job', 'gehabt', 'Z', 'und', 'zuletzt', 'arbeite', 'von', '_', 'NUMBER', 'bis', 'Telekaffe', 'Gottest', 'Geseck', 'wenn', 'Sie', 'riserviren', 'gerne', 'wissen', 'ab', 'wann', 'Beginn', 'dieser', 'Ausbildung', 'wie', 'viele', 'Stunden', 'Prowoche', 'Arbeitszeit', 'vorher', 'Ihn', 'Dankber', 'züruck', 'schreiben', 'können', 'Viellen', 'Dank', 'viellen', 'Grüßen', 'Frau', 'MeierAnlage']


In [23]:
import textacy
from textacy import preprocessing

In [20]:
textacy.load_spacy_lang("de")

In [24]:
text = textacy.preprocessing.remove.remove_punctuation(text)

Morgen geht es mir nicht so gut 
